In [8]:
import os
import numpy as np
import xarray as xr
from pystac_client import Client

# Path to the directory containing NetCDF files
netcdf_directory = "C:/Users/r134p694_a/Training Netcdf"

# STAC API client setup
client = Client.open("https://catalogue.dataspace.copernicus.eu/stac") 
collection = "SENTINEL-1"

# Initialize an empty list to collect matching filtered items
matching_items = []

# Iterate through each NetCDF file in the directory
for netcdf_file in os.listdir(netcdf_directory):
    if netcdf_file.endswith('.nc'):
        # Read the NetCDF file
        ds = xr.open_dataset(os.path.join(netcdf_directory, netcdf_file))
        
        # Calculate the bounding box
        lat_min = np.min(ds.sar_grid2d_latitude.values)
        lat_max = np.max(ds.sar_grid2d_latitude.values)
        lon_min = np.min(ds.sar_grid2d_longitude.values)
        lon_max = np.max(ds.sar_grid2d_longitude.values)

        # Define the bounding box (min_lon, min_lat, max_lon, max_lat)
        bbox = [lon_min, lat_min, lon_max, lat_max]

        # Extract the date from the scene ID in attributes (if exists)
        scene_id = ds.attrs.get('scene_id', '')  # Adjust based on actual attribute name
        date = scene_id[:8] if scene_id else ''  # Extract date part

        # Perform the Search with bbox and datetime
        search = client.search(
            collections=[collection],
            bbox=bbox,
            datetime=f"{date[:4]}-{date[4:6]}-{date[6:8]}",
            limit=100
        )

        # Fetch All Matching Items
        try:
            items = list(search.get_items())
            print(f"Found {len(items)} items matching bbox and datetime criteria for {netcdf_file}.")
        except Exception as e:
            print(f"An error occurred during the search: {e}")
            continue

        # Initialize an empty list to store filtered items
        filtered_items = []

        # Iterate through retrieved items and filter by 's1:instrument_mode' and 'processingLevel'
        for item in items:
            instrument_mode = item.properties.get('operationalMode', '')
            processing_level = item.properties.get('processingLevel', '')
            
            # Check if both conditions are met
            if instrument_mode == 'EW' and processing_level == 'LEVEL1':
                filtered_items.append(item)

        print(f"Filtered down to {len(filtered_items)} EW mode and LEVEL1 items for {netcdf_file}.")

        # Get the original ID from the dataset attributes
        original_id = ds.attrs.get('original_id', '')  # Adjust based on actual attribute name
        # Extract the part of original_id before '_icechart'
        original_id_prefix = original_id.split('_icechart')[0] if '_icechart' in original_id else original_id

        # Match the original_id with filtered items
        for filtered_item in filtered_items:
            filtered_id = filtered_item.id.split('.')[0]  # Remove the ".SAFE" part
            if original_id_prefix in filtered_id:  # Check for a match
                matching_items.append(filtered_item)
                break  # Stop after finding the first match

# Output the matched items
print(f"Total matching items collected: {len(matching_items)}")
for item in matching_items:
    print(item.id)


Found 39 items matching bbox and datetime criteria for 20180108T184332_dmi_prep.nc.
Filtered down to 20 EW mode and LEVEL1 items for 20180108T184332_dmi_prep.nc.
Found 43 items matching bbox and datetime criteria for 20180306T101622_dmi_prep.nc.
Filtered down to 18 EW mode and LEVEL1 items for 20180306T101622_dmi_prep.nc.
Found 6 items matching bbox and datetime criteria for 20180311T111524_cis_prep.nc.
Filtered down to 4 EW mode and LEVEL1 items for 20180311T111524_cis_prep.nc.
Found 32 items matching bbox and datetime criteria for 20180325T194759_dmi_prep.nc.
Filtered down to 10 EW mode and LEVEL1 items for 20180325T194759_dmi_prep.nc.
Found 9 items matching bbox and datetime criteria for 20180707T122209_cis_prep.nc.
Filtered down to 6 EW mode and LEVEL1 items for 20180707T122209_cis_prep.nc.
Found 32 items matching bbox and datetime criteria for 20180721T120156_cis_prep.nc.
Filtered down to 24 EW mode and LEVEL1 items for 20180721T120156_cis_prep.nc.
Found 24 items matching bbox and

### Copernicus dataspace Session

In [15]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Your client credentials
client_id = '###'
client_secret = '###'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret, include_client_id=True)

def sentinelhub_compliance_hook(response):
    response.raise_for_status()
    return response

oauth.register_compliance_hook("access_token_response", sentinelhub_compliance_hook)
# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
print(resp.content)



b'[]'


## keycloak token

In [16]:
import requests
def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
        )
    return r.json()["access_token"]

keycloak_token = get_keycloak('###','###')
keycloak_token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJYVUh3VWZKaHVDVWo0X3k4ZF8xM0hxWXBYMFdwdDd2anhob2FPLUxzREZFIn0.eyJleHAiOjE3MjgwNzAzNjksImlhdCI6MTcyODA2OTc2OSwianRpIjoiZTBkZmM4N2QtMmY0Ni00ZjNlLTk0ZTAtNTBiMDU0MzhmNDMxIiwiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS5kYXRhc3BhY2UuY29wZXJuaWN1cy5ldS9hdXRoL3JlYWxtcy9DRFNFIiwiYXVkIjpbIkNMT1VERkVSUk9fUFVCTElDIiwiYWNjb3VudCJdLCJzdWIiOiIyYWMyNDIwZS1kM2MwLTQyNWYtYTY2OS1jYWY4YjA3N2QxZjMiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJjZHNlLXB1YmxpYyIsInNlc3Npb25fc3RhdGUiOiJjZmI1MzY1NS05NDU2LTRjMjQtYmFjZi00N2I4NGU3MTVlZjQiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cHM6Ly9sb2NhbGhvc3Q6NDIwMCIsIioiLCJodHRwczovL3dvcmtzcGFjZS5zdGFnaW5nLWNkc2UtZGF0YS1leHBsb3Jlci5hcHBzLnN0YWdpbmcuaW50cmEuY2xvdWRmZXJyby5jb20iXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLWNkYXMiLCJjb3Blcm5pY3VzLWdlbmVyYWwiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6IkFVREl

In [18]:
import os
import requests

# Ensure the directory exists
output_dir = 'C:/Users/r134p694_a/Cop_Matching/'
#os.makedirs(output_dir, exist_ok=True)

# Function to download the item
def download_item(item):
    keycloak_token = get_keycloak('###','###')
    # Access the 'PRODUCT' asset to get the download link
    product_asset = item.assets.get('PRODUCT')
    
    if product_asset:
        # Extract the product ID from the asset URL
        product_id = product_asset.href.split('(')[1].split(')')[0]  # Extract the product ID
        
        # Construct the download URL for the ZIP file
        download_url = f'https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value'

        print(f"Downloading from: {download_url}")
        
        # Optional: Use headers for authorization if needed
        headers = {
            'Authorization': f"Bearer {keycloak_token}"  # Replace with your actual token if needed
        }
        
        # Download the file
        response = requests.get(download_url, headers=headers)
        
        if response.status_code == 200:
            # Save the file as a ZIP
            file_name = os.path.join(output_dir, f"{item.id}.zip")  # Change the extension to .zip
            with open(file_name, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {file_name}")
        else:
            print(f"Failed to download {item.id} with status code: {response.status_code}")
            print(f"Error Message: {response.text if response.text else 'No additional error message'}")
    else:
        print(f"No PRODUCT asset found for {item.id}")

#Iterate through filtered items and download each
for item in matching_items:
    download_item(item)


Downloaded: C:/Users/r134p694_a/Cop_Matching/S1A_EW_GRDM_1SDH_20180108T184332_20180108T184437_020067_022330_93EB.SAFE.zip
Downloaded: C:/Users/r134p694_a/Cop_Matching/S1A_EW_GRDM_1SDH_20180306T101622_20180306T101722_020893_023D76_206D.SAFE.zip
Downloaded: C:/Users/r134p694_a/Cop_Matching/S1B_EW_GRDM_1SDH_20180311T111524_20180311T111623_009983_01218C_F0B7.SAFE.zip
Downloaded: C:/Users/r134p694_a/Cop_Matching/S1A_EW_GRDM_1SDH_20180325T194759_20180325T194859_021176_024676_449C.SAFE.zip
Downloaded: C:/Users/r134p694_a/Cop_Matching/S1A_EW_GRDM_1SDH_20180707T122209_20180707T122309_022688_027552_A48E.SAFE.zip
Downloaded: C:/Users/r134p694_a/Cop_Matching/S1A_EW_GRDM_1SDH_20180721T120156_20180721T120256_022892_027B98_56D9.SAFE.zip
Downloaded: C:/Users/r134p694_a/Cop_Matching/S1B_EW_GRDM_1SDH_20180829T113858_20180829T114003_012477_017029_869A.SAFE.zip
Downloaded: C:/Users/r134p694_a/Cop_Matching/S1B_EW_GRDM_1SDH_20180902T124300_20180902T124400_012536_017204_4133.SAFE.zip
Downloaded: C:/Users/r13

## Save as netcdfs

In [21]:
import os
import zipfile
import xarray as xr
import rasterio

# Paths
source_dir = r'C:/Users/r134p694_a/Cop_Matching/'
netcdf_dir = r'C:/Users/r134p694_a/Cop_Matching/netcdfs/'

# Ensure the NetCDF directory exists
os.makedirs(netcdf_dir, exist_ok=True)

# Function to add TIFF data as a dataset in a NetCDF file
def add_tiff_to_netcdf(tiff_path, ncfile, variable_name):
    with rasterio.open(tiff_path) as src:
        data = src.read(1)  # Read the first band
        coords = {
            'y': np.arange(data.shape[0]),
            'x': np.arange(data.shape[1])
        }
        da = xr.DataArray(data, dims=['y', 'x'], coords=coords, name=variable_name)
        da.to_netcdf(ncfile, mode='a')  # Append to existing NetCDF file

# Loop through all ZIP files
for file_name in os.listdir(source_dir):
    if file_name.endswith('.zip'):
        zip_path = os.path.join(source_dir, file_name)
        base_name = os.path.splitext(file_name)[0]  # Name without extension
        
        # Create NetCDF file for each ZIP file
        netcdf_path = os.path.join(netcdf_dir, f"{base_name}.nc")

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            hh_file = None
            hv_file = None
            for zip_info in zip_ref.infolist():
                # Only process files in the "measurement" folder
                if 'measurement' in zip_info.filename and zip_info.filename.endswith('.tiff'):
                    # Check if the file name contains "hh" or "hv"
                    if 'hh' in zip_info.filename.lower():
                        hh_file = zip_info.filename
                    elif 'hv' in zip_info.filename.lower():
                        hv_file = zip_info.filename

            # Extract and add the hh file to NetCDF
            if hh_file:
                hh_tiff_path = zip_ref.extract(hh_file, source_dir)
                add_tiff_to_netcdf(hh_tiff_path, netcdf_path, 'hh')
                os.remove(hh_tiff_path)  # Clean up extracted file

            # Extract and add the hv file to NetCDF
            if hv_file:
                hv_tiff_path = zip_ref.extract(hv_file, source_dir)
                add_tiff_to_netcdf(hv_tiff_path, netcdf_path, 'hv')
                os.remove(hv_tiff_path)  # Clean up extracted file

print("Tiff files have been saved in NetCDF files!")


Tiff files have been saved in NetCDF files!


In [31]:
import os
import xarray as xr
import numpy as np

# Path to original and downsampled NetCDF files
netcdf_dir = r'C:/Users/r134p694_a/Cop_Matching/netcdfs/'
downsampled_dir = r'C:/Users/r134p694_a/Cop_Matching/netcdfs/downsampled/'

# Ensure the downsampled directory exists
os.makedirs(downsampled_dir, exist_ok=True)

# Downsample function
def downsample_image(image, scale_factor=2):
    """Downsamples the image using a 2x2 averaging kernel."""
    if image.shape[0] % scale_factor != 0 or image.shape[1] % scale_factor != 0:
        image = image[:image.shape[0] - image.shape[0] % scale_factor,
                      :image.shape[1] - image.shape[1] % scale_factor]

    downsampled_image = image.reshape(
        (image.shape[0] // scale_factor, scale_factor, 
         image.shape[1] // scale_factor, scale_factor)).mean(axis=(1, 3))
    
    return downsampled_image

# Loop through all NetCDF files
for file_name in os.listdir(netcdf_dir):
    if file_name.endswith('.nc'):
        netcdf_path = os.path.join(netcdf_dir, file_name)
        downsampled_netcdf_path = os.path.join(downsampled_dir, file_name)  # Save to downsampled folder
        
        try:
            # Open the original NetCDF file in read-only mode
            with xr.open_dataset(netcdf_path, mode='r') as ds:
                # Create a new dataset for downsampled data
                downsampled_ds = xr.Dataset()

                # Downsample and save the 'hh' and 'hv' datasets
                if 'hh' in ds.variables:
                    hh_image = ds['hh'].values  # Extract original image data
                    downsampled_hh = downsample_image(hh_image)  # Downsample it
                    # Create a new variable in the downsampled dataset with downsampled data
                    downsampled_ds['hh'] = (('y', 'x'), downsampled_hh)

                if 'hv' in ds.variables:
                    hv_image = ds['hv'].values  # Extract original image data
                    downsampled_hv = downsample_image(hv_image)  # Downsample it
                    # Create a new variable in the downsampled dataset with downsampled data
                    downsampled_ds['hv'] = (('y', 'x'), downsampled_hv)

                # Carry over the attributes from the original dataset (optional)
                downsampled_ds.attrs = ds.attrs

                # Optionally, copy over other metadata or dimensions if needed
                downsampled_ds['x'] = (('x',), np.arange(downsampled_hh.shape[1]))
                downsampled_ds['y'] = (('y',), np.arange(downsampled_hh.shape[0]))

                # Save the downsampled dataset to the new NetCDF file
                downsampled_ds.to_netcdf(downsampled_netcdf_path)
                print(f"Downsampled file saved as: {downsampled_netcdf_path}")
                
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")

print("Downsampling completed and saved in the downsampled directory.")


Downsampled file saved as: C:/Users/r134p694_a/Cop_Matching/netcdfs/downsampled/S1A_EW_GRDM_1SDH_20180108T184332_20180108T184437_020067_022330_93EB.SAFE.nc
Downsampled file saved as: C:/Users/r134p694_a/Cop_Matching/netcdfs/downsampled/S1A_EW_GRDM_1SDH_20180306T101622_20180306T101722_020893_023D76_206D.SAFE.nc
Downsampled file saved as: C:/Users/r134p694_a/Cop_Matching/netcdfs/downsampled/S1A_EW_GRDM_1SDH_20180325T194759_20180325T194859_021176_024676_449C.SAFE.nc
Downsampled file saved as: C:/Users/r134p694_a/Cop_Matching/netcdfs/downsampled/S1A_EW_GRDM_1SDH_20180707T122209_20180707T122309_022688_027552_A48E.SAFE.nc
Downsampled file saved as: C:/Users/r134p694_a/Cop_Matching/netcdfs/downsampled/S1A_EW_GRDM_1SDH_20180721T120156_20180721T120256_022892_027B98_56D9.SAFE.nc
Downsampled file saved as: C:/Users/r134p694_a/Cop_Matching/netcdfs/downsampled/S1A_EW_GRDM_1SDH_20180914T115342_20180914T115442_023694_02952F_CA97.SAFE.nc
Downsampled file saved as: C:/Users/r134p694_a/Cop_Matching/netc